# THE BATTLE OF CITIES

---------------

### PART 1

---------------

#### 1. A description of the problem and a discussion of the background.

As a traveler, I want to know where to plan my next trip. So I decided to make a city ranking. In this ranking will be included the global score of the city, the score of touring (tourism), relaxation, sociability and sportiveness. Every tourist wants to know if the next city break that they will make will have everything they need and they want to know that broadly speaking, not doing so much research to begin with.       

#### 2. A description of the data and how it will be used to solve the problem.

How I will be able to compute the scores? Well, all these scores are covered by data gathered from Foursquare API. I can count the number of hotels/motels, airports, museums/art galleries/monuments that are in that city in a 20km range (we want to cover even the biggest city) and compute the tourism score for that city by adding a weight to every kind of place. Same goes for the other scores. A global score will have the same principle, but will be formed by the scores from tourism, relaxation, sociability and sportiveness (with their corresponding weights as well).

---------------

### PART 2

---------------